In [2]:
import yfinance as yf
import pandas as pd
from datetime import date

Want to implement a JS version so can open in browser w/ D3 vis etc

Maybe inherit from some Market class to automatically grab prices – Or can just grab prices directly from API like yfinance
- The more accurate the better; yfinance maybe not the move

TODO: export it (offline) and then figure out how to save it online
- Maybe have a hardcore mode where funds actually matter (can't buy without enough)

In [146]:
class Portfolio:

    def __init__(self, funds):
        self.funds = funds
        self.fund_hist = [funds]
        self.portfolio = {}

    def getPrice(self, ticker, day):
        prices = yf.download(ticker, day)
        price = data.Close[0] # using an arbitrary price for now 
        return price
        
    def buy(self, ticker, num_shares, day = None):
        """Buys share(s).  If no date specified, assumes current day"""
        if day is None:
            day = str(date.today())

        price = self.getPrice(ticker, day)
        
        if ticker in self.portfolio.keys():
            self.portfolio[ticker].append(('BUY', day, num_shares, price))
            prev_shares = self.portfolio[ticker][0]
            prev_total = self.portfolio[ticker][1]
            self.portfolio[ticker][0] = prev_shares + num_shares
            self.portfolio[ticker][1] = prev_total + num_shares*price
        else:
            self.portfolio.update({ticker:[num_shares, num_shares*price, ('BUY', day, num_shares, price)]})
        
        self.funds = self.funds - num_shares*price
        self.fund_hist.append(self.funds)
    
    def sell(self, ticker, num_shares, day = None):

        if day is None:
            day = str(date.today())
        
        price = self.getPrice(ticker, day)  
        
        if ticker not in self.portfolio.keys():
            raise KeyError('Ticker not found in portfolio.')
        
        else:
            prev_shares = self.portfolio[ticker][0]
            prev_total = self.portfolio[ticker][1]
            if num_shares >  self.portfolio[ticker][1]:
                raise ValueError('Size {} sell order > {} current shares'.format(num_shares, prev_shares))
            else:
                self.portfolio[ticker].append(('SELL', day, num_shares, price))
                self.portfolio[ticker][0] = prev_shares - num_shares
                self.portfolio[ticker][1] = prev_total - num_shares*price
                
            
        self.funds = self.funds + num_shares*price
        self.fund_hist.append(self.funds)
        
    def display(self, mode = 'df'):
        if mode == 'df':
            df = pd.DataFrame.from_dict(self.portfolio, orient='index', columns = ['Shares', 'Equity', 'H'])
            return df.drop('H', 1)
        elif mode == 'hist':
            print(self.portfolio)

In [147]:
brain_fund = Portfolio(1000000)

In [148]:
brain_fund.buy('AAPL', 400)
brain_fund.display('hist')

[*********************100%***********************]  1 of 1 completed
{'AAPL': [400, 396715.9912109375, ('BUY', '2020-06-17', 400, 991.7899780273438)]}


In [149]:
brain_fund.buy('TSLA',500)

[*********************100%***********************]  1 of 1 completed


In [150]:
d = brain_fund.display('df')
d

,Shares,Equity
AAPL,400,396715.991211
TSLA,500,495894.989014


In [6]:
data = yf.download('$TSLA', '2020-07-13')


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- $TSLA: No data found, symbol may be delisted


In [5]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
